# Mapping All Fatal Collisions in NYC

___NYC residents and visitors may be surprised to see how many fatal collisions have occurred where they live, work, and travel since October 2012___

### Definitions
- __Collision:__ A motor vehicle collision involving injuries, deaths, or a significant amount of property damage (~> $1000) reported on a New York State form, MV104-AN
- __Fatal Collision:__ A collision where at least one person is killed

### Data Sources

- Collision data obtained from https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95
- Data was processed by running `process_raw.data.py` which saves processed data to local directory specified in `process_raw.data.py` script
- _Note that \~12% of collisions are missing lat-long coordinates. These collisions are excluded from the following geographic analyses._

In [1]:
import os.path
import folium
import pandas as pd
from src import visualizations as viz

# Parameters

In [2]:
PROCESSED_CRASH_DATA = "data/processed/crashes.pkl"
IMG_DIR = "website/where/overall_maps"
NYC_MAP_CENTER = (40.73, -73.92)

In [3]:
crashes = pd.read_pickle(PROCESSED_CRASH_DATA)

# Map of all Serious Collisions

_Zoom in to see serious collisions and the date, time, number injured, and number killed_

In [5]:
fatal = crashes[crashes["valid_lat_long"] & (crashes["KILLED"] > 0)]
map_data = fatal[["LAT", "LONG", "DATE", "TIME", "INJURED", "KILLED"]]
fatal_map = folium.Map(location=NYC_MAP_CENTER, zoom_start=10, tiles="OpenStreetMap")
js_callback = (
    "function (row) {"
    "var marker = L.marker(new L.LatLng(row[0], row[1]));"
    "var icon = L.AwesomeMarkers.icon({"
    "icon: 'fa-exclamation',"
    "iconColor: 'white',"
    "markerColor: 'red',"
    "prefix: 'fa',"
    "});"
    "marker.setIcon(icon);"
    "var popup = L.popup({maxWidth: '300'});"
    "const display_text = {text:"
    "'DATE: ' + row[2] + '<br>' +"
    "'TIME: ' + row[3] + '<br>' +"
    "'PEOPLE INJURED: ' + row[4] + '<br>' +"
    "'PEOPLE KILLED: ' + row[5]};"
    "var poptext = $(`<div id='mytext' style='width: 100.0%; height: 100.0%;'> ${display_text.text}</div>`)[0];"
    "popup.setContent(poptext);"
    "marker.bindPopup(popup);"
    "return marker};"
)
folium.plugins.FastMarkerCluster(data=map_data, callback=js_callback).add_to(
    fatal_map
)
fatal_map.save(os.path.join(IMG_DIR, "all_fatal_map.html"))
fatal_map